In [20]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go

In [21]:
def fit(x,y):
    A = np.vstack([x, np.ones(len(x))]).T
    a,b = np.linalg.lstsq(A, y, rcond=None)[0]
    return a,b

# Работа 5.1
# Измерение коэффциента ослабления потока $\gamma$-лучей в веществе и определение их энергии

С помощью сцинтилляционного счетчика измеряются линейные коэффициенты ослабления потока $\gamma$-лучей в свинце, железе и алюминии; по их величине определяется энергия $\gamma$-квантов.
Гамма-лучи возникают при переходе возбужденных ядер из одного
энергетического состояния в другое, более низкое. Энергия $\gamma$-квантов обычно заключена между несколькими десятками килоэлектронвольт и несколькими миллионами электрон-вольт. Гамма-кванты не несут электрического заряда, их масса равна нулю. Проходя через вещество, пучок $\gamma$-квантов постепенно ослабляется. Ослабление происходит по экспоненциальному закону, который может быть записан в
двух эквивалентных формах:

\begin{equation}
I = I_0e^{\mu l}
\end{equation}

\begin{equation}
I = I_0e^{\mu' m_1}
\end{equation}

$I,I_0$ - интенсивности прошедшего и падающего излучений
$l$ - длина пути, пройденного $\gamma$ - лучей, $\mu, \mu'$.

Тогда $\mu$ определяется:
$$
\mu = \frac{1}{l}ln\frac{N_0}{N}
$$

# Задание
1. Включите пересчетный прибор и высоковольтный выпрямитель. Дайте им прогреться в течение 5-10 минут
2. Убедитесь в том, что установка чувствует $\gamma$-лучи. Для этого подайте на ФЭУ напряжение, указанное на установке. Измерьте скорость счета при полностью открытом коллиматоре, а затем при коллиматоре, закрытом свинцовой пробкой. Скорость счета должна резко уменьшиться. Чтобы убедиться в работоспособности установки, повторите эту процедуру 23 раза.

3. При вычислении $N_0$ и $N$ из показаний прибора необходимо вычитать он, который обусловлен шумом ФЭУ и посторонними частицами: космическим излучением, $\gamma$-квантами от соседних источников, квантами, рассеянными на стенах комнаты и в стенках прибора и т.д.

In [22]:
data_0 = np.array([[95429,210],
[94740,229],
[95331,189],
[95167,209]])
data = pd.DataFrame(data_0, 
                    index = np.arange(1,len(data_0)+1,1),
                    columns = ['$N_0 $ без преграды','шум'])

data = data.append({'$N_0 $ без преграды':np.mean(data_0[:,0]),'шум':np.mean(data_0[:,1])}, ignore_index=True)
data = data.rename(index = {len(data)-1:'Средняя'})
N0 = data.iloc[4,0]-data.iloc[4,1]
data

,$N_0 $ без преграды,шум
0,95429.00,210.00
1,94740.00,229.00
2,95331.00,189.00
3,95167.00,209.00
Средняя,95166.75,209.25


In [23]:
print('N_0 без шума','=',N0)

N_0 без шума = 94957.5


In [24]:
def create_table (data,lenth_of_material,average_value_without_noize):
    table = pd.DataFrame({'lenth':np.arange(lenth_of_material,(len(data)+1)*lenth_of_material,lenth_of_material),
                         'data_0':data},
                         index = np.arange(1,len(data)+1,1))
    
    table = table.assign(data_1 = lambda x: x.data_0 - 209)
    data_2 = pd.Series(index = np.arange(1,len(data)+1,1))
    
    for i in range(len(data)):
        data_2[i+1] = math.log(average_value_without_noize/table['data_1'][i+1])

    table['ln(N)'] = data_2
    
    mu = pd.Series(index = np.arange(1,len(data)+1,1))
    for i in range(len(data)):
        mu[i+1] = 1/table['lenth'][i+1]*math.log(average_value_without_noize/table['data_1'][i+1])
    table['$\mu$'] = mu
    table = table.rename(columns = {'lenth':'Длина(cm)','data_0':'N с шумом','data_1':'N без шума'})
    return table

In [25]:
lenth = pd.DataFrame ({'Материал':['Алюминий','Свинец','Железо','Дерево'],
                       'Длина(cm)':[2,0.45,1,2]},
                     index = np.arange(1,5,1))
lenth

,Длина(cm),Материал
1,2.00,Алюминий
2,0.45,Свинец
3,1.00,Железо
4,2.00,Дерево


### Данные

1. Аллюминий

In [26]:
all = create_table([
    61200,
    41053,
    27267,
    18350,
    12489,
    8583,
    5764],2,N0)
all

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$
1,61200,2,60991,0.442703,0.221352
2,41053,4,40844,0.843669,0.210917
3,27267,6,27058,1.255447,0.209241
4,18350,8,18141,1.655255,0.206907
5,12489,10,12280,2.045458,0.204546
6,8583,12,8374,2.428298,0.202358
7,5764,14,5555,2.838731,0.202766


In [27]:
mu_all_aver = np.mean(all.iloc[:,4])
mu_all_delta = all.iloc[:,4] - mu_all_aver
all['$\delta \mu$'] = mu_all_delta
print('mu_all_aver =',mu_all_aver)
print('Cреднеквадратичное отклонение =',np.std(all.iloc[:,4], ddof=1))

mu_all_aver = 0.2082981857167199
Cреднеквадратичное отклонение = 0.0065833110872393075


In [28]:
all

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$,$\delta \mu$
1,61200,2,60991,0.442703,0.221352,0.013053
2,41053,4,40844,0.843669,0.210917,0.002619
3,27267,6,27058,1.255447,0.209241,0.000943
4,18350,8,18141,1.655255,0.206907,-0.001391
5,12489,10,12280,2.045458,0.204546,-0.003752
6,8583,12,8374,2.428298,0.202358,-0.005940
7,5764,14,5555,2.838731,0.202766,-0.005532


2. Свинец

In [29]:
pb =  create_table([
    54145,
    32575,
    18100,
    10354,
    6213,
    4185,
    2504,
    1666,
    1152],0.45,N0)
pb

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$
1,54145,0.45,53936,0.565631,1.256958
2,32575,0.90,32366,1.076321,1.195912
3,18100,1.35,17891,1.669132,1.236394
4,10354,1.80,10145,2.236448,1.242471
5,6213,2.25,6004,2.761004,1.227113
6,4185,2.70,3976,3.173153,1.175242
7,2504,3.15,2295,3.722697,1.181808
8,1666,3.60,1457,4.177050,1.160292
9,1152,4.05,943,4.612118,1.138795


In [30]:
mu_pb_aver = np.mean(pb.iloc[:,4])
mu_pb_delta = pb.iloc[:,4] - mu_pb_aver
pb['$\delta \mu$'] = mu_pb_delta
print('mu_pb_aver =',mu_pb_aver)
print('Cреднеквадратичное отклонение =',np.std(pb.iloc[:,4], ddof=1))

mu_pb_aver = 1.2016649994623623
Cреднеквадратичное отклонение = 0.040875085150684935


In [31]:
pb

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$,$\delta \mu$
1,54145,0.45,53936,0.565631,1.256958,0.055293
2,32575,0.90,32366,1.076321,1.195912,-0.005753
3,18100,1.35,17891,1.669132,1.236394,0.034729
4,10354,1.80,10145,2.236448,1.242471,0.040806
5,6213,2.25,6004,2.761004,1.227113,0.025448
6,4185,2.70,3976,3.173153,1.175242,-0.026423
7,2504,3.15,2295,3.722697,1.181808,-0.019857
8,1666,3.60,1457,4.177050,1.160292,-0.041373
9,1152,4.05,943,4.612118,1.138795,-0.062870


3. Железо

In [32]:
fe =  create_table([
    52100,
    29492,
    16494,
    9744,
    5342,
    3562,
    2273,
    1358,
    879],1,N0)
fe

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$
1,52100,1,51891,0.604284,0.604284
2,29492,2,29283,1.176422,0.588211
3,16494,3,16285,1.763185,0.587728
4,9744,4,9535,2.298460,0.574615
5,5342,5,5133,2.917739,0.583548
6,3562,6,3353,3.343574,0.557262
7,2273,7,2064,3.828784,0.546969
8,1358,8,1149,4.414537,0.551817
9,879,9,670,4.953907,0.550434


In [33]:
mu_fe_aver = np.mean(fe.iloc[:,4])
mu_fe_delta = fe.iloc[:,4] - mu_fe_aver
fe['$\delta \mu$'] = mu_fe_delta
print('mu_fe_aver =',mu_fe_aver)
print('Cреднеквадратичное отклонение =',np.std(fe.iloc[:,4], ddof=1))

mu_fe_aver = 0.5716521218463746
Cреднеквадратичное отклонение = 0.020638929274498202


In [34]:
fe

,N с шумом,Длина(cm),N без шума,ln(N),$\mu$,$\delta \mu$
1,52100,1,51891,0.604284,0.604284,0.032632
2,29492,2,29283,1.176422,0.588211,0.016559
3,16494,3,16285,1.763185,0.587728,0.016076
4,9744,4,9535,2.298460,0.574615,0.002963
5,5342,5,5133,2.917739,0.583548,0.011896
6,3562,6,3353,3.343574,0.557262,-0.014390
7,2273,7,2064,3.828784,0.546969,-0.024683
8,1358,8,1149,4.414537,0.551817,-0.019835
9,879,9,670,4.953907,0.550434,-0.021218


In [35]:
all_cof = fit(all.iloc[:,1],all.iloc[:,3])
pb_cof = fit(pb.iloc[:,1],pb.iloc[:,3])
fe_cof = fit(fe.iloc[:,1],fe.iloc[:,3])
cof = [all_cof,pb_cof,fe_cof]
all_x = np.linspace(min(all.iloc[:,1]),max(all.iloc[:,1]),100)
pb_x = np.linspace(min(pb.iloc[:,1]),max(pb.iloc[:,1]),100)
fe_x = np.linspace(min(fe.iloc[:,1]),max(fe.iloc[:,1]),100)

In [40]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = all.iloc[:,1],y=all.iloc[:,3],
                    mode='markers',
                             marker=dict(color='red'),
                         error_y=dict(
                            type='constant',
                            value=np.std(all.iloc[:,4], ddof=1)),
                    name='Аллюминий'))
fig.add_trace(go.Scatter(x = all_x,y=all_x*all_cof[0]+all_cof[1],
                             line=dict(color='red'),showlegend=False))
              
fig.add_trace(go.Scatter(x = pb.iloc[:,1],y=pb.iloc[:,3],
                    mode='markers',
                             marker=dict(color='green'),
                         error_y=dict(
                            type='constant',
                            value=np.std(pb.iloc[:,4], ddof=1)),
                    name='Свинец'))

fig.add_trace(go.Scatter(x = pb_x,y=pb_x*pb_cof[0]+pb_cof[1],
                             line=dict(color='green'),showlegend=False))
              
fig.add_trace(go.Scatter(x = fe.iloc[:,1],y=fe.iloc[:,3],
                    mode='markers',
                             marker=dict(color='blue'),
                         error_y=dict(
                            type='constant',
                            value=np.std(fe.iloc[:,4], ddof=1)),
                    name='Железо'))
fig.add_trace(go.Scatter(x = fe_x,y=fe_x*fe_cof[0]+fe_cof[1],
                             line=dict(color='blue'),showlegend=False))
fig.update_layout( xaxis_title='Длина(cm)',
                   yaxis_title='ln(N)')
fig.show()
fig.write_image("images/fig1.pdf")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    /Users/pavel/anaconda3/bin
    /usr/local/bin
    /usr/bin
    /bin
    /usr/sbin
    /sbin
    /Library/TeX/texbin
    /usr/local/share/dotnet
    /Library/Frameworks/Mono.framework/Versions/Current/Commands
    /Applications/Xamarin Workbooks.app/Contents/SharedSupport/path-bin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orca

Alternatively, see other installation methods in the orca project README at
https://github.com/plotly/orca.

After installation is complete, no further configuration should be needed.

If you have installed orca, then for some reason plotly.py was unable to
locate it. In this case, set the `plotly.io.orca.config.executable`
property to the full path of your orca executable. For example:

    >>> plotly.io.orca.c

In [37]:
mat = ['Аллюминий','Свинец','Железо']
for i in range(3):
    print('Ln(N/N0)/l',mat[i],'=',round(cof[i][0],3))

Ln(N/N0)/l Аллюминий = 0.199
Ln(N/N0)/l Свинец = 1.131
Ln(N/N0)/l Железо = 0.538


In [38]:
mat_ro = [2.7, 11.34, 7.874]
for i in range(3):
    print('Ln(N/N0)/l',mat[i],'=',round(cof[i][0],3)/ mat_ro[i])

Ln(N/N0)/l Аллюминий = 0.0737037037037037
Ln(N/N0)/l Свинец = 0.09973544973544973
Ln(N/N0)/l Железо = 0.06832613665227331
